# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
from keras import backend as K

class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename)
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['text_cleaned'], float):
                texts.append(str(row['text_cleaned']))
            else:
                texts.append(row['text_cleaned'])

            labels.append(row['target'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = tf.keras.preprocessing.sequence.pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_testing_metric(self, y_test, y_pred):
        accuracyScore = accuracy_score(y_test, y_pred)
        precisionScore = precision_score(y_test, y_pred)
        recallScore = recall_score(y_test, y_test)
        f1Score = f1_score(y_test, y_pred)
        return accuracyScore, precisionScore, recallScore, f1Score

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_val_data(self, textsTraining, labelsTraining, train_index, val_index, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining)[train_index], np.asarray(textsTraining)[val_index]
        y_train, y_val = np.asarray(labelsTraining)[train_index], np.asarray(labelsTraining)[val_index]

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)

    def recall(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_score(self, y_true, y_pred):
        precision = self.precision(y_true, y_pred)
        recall = self.recall(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


# CNN

In [2]:
import tensorflow as tf

class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        util = utility()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[util.f1_score])

        return model

# Path

In [3]:
import os 
# path

training_path = 'trainPreprocessed.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = '{}/Dataset/Disaster Tweets/'.format(root_path)
resultsPath = '{}/Server2/Disaster Tweets/Results/'.format(root_path)
archPath = '{}/Server2/Disaster Tweets/Architecture/'.format(root_path)
testing_name = "Experiment1_GridSearch"
glovePath = ['{}/Glove/glove.6B.50d.txt'.format(root_path),
             '{}/Glove/glove.6B.100d.txt'.format(root_path),
             '{}/Glove/glove.6B.200d.txt'.format(root_path),
             '{}/Glove/glove.6B.300d.txt'.format(root_path)]

# Run

In [4]:
import time
import os
import csv
import tensorflow as tf

# Optimized hyperparameters
epochs = [20,40,60,80,100]
batch_size = [64,256]
optimizer = ['adam', 'nadam', 'adamax', 'adagrad', 'rmsprop']
learning_rate = [0.001,0.01]
layers=[]

combination = [[h, i, j, k, l] for h in epochs
       for i in batch_size
       for j in optimizer
       for k in learning_rate
       for l in layers]

# Fixed hyperparameters
hyperparameters = {}
hyperparameters['output_dim'] = 100
hyperparameters['momentum'] = 0.9
hyperparameters['num_filters1'] = 64
hyperparameters['num_filters2'] = 64
hyperparameters['num_filters3'] = 64
hyperparameters['num_filters4'] = 64
hyperparameters['num_filters5'] = 64
hyperparameters['num_filters6'] = 64
hyperparameters['num_filters7'] = 64
hyperparameters['num_filters8'] = 64
hyperparameters['num_filters9'] = 64
hyperparameters['num_filters10'] = 64
hyperparameters['num_filters11'] = 64
hyperparameters['num_filters12'] = 64
hyperparameters['num_filters13'] = 64
hyperparameters['num_filters14'] = 64
hyperparameters['num_filters15'] = 64
hyperparameters['kernel_size1'] = 3
hyperparameters['kernel_size2'] = 3
hyperparameters['kernel_size3'] = 3
hyperparameters['kernel_size4'] = 3
hyperparameters['kernel_size5'] = 3
hyperparameters['kernel_size6'] = 3
hyperparameters['kernel_size7'] = 3
hyperparameters['kernel_size8'] = 3
hyperparameters['kernel_size9'] = 3
hyperparameters['kernel_size10'] = 3
hyperparameters['kernel_size11'] = 3
hyperparameters['kernel_size12'] = 3
hyperparameters['kernel_size13'] = 3
hyperparameters['kernel_size14'] = 3
hyperparameters['kernel_size15'] = 3
hyperparameters['conv_activation_func1'] = 'relu'
hyperparameters['conv_activation_func2'] = 'relu'
hyperparameters['conv_activation_func3'] = 'relu'
hyperparameters['conv_activation_func4'] = 'relu'
hyperparameters['conv_activation_func5'] = 'relu'
hyperparameters['conv_activation_func6'] = 'relu'
hyperparameters['conv_activation_func7'] = 'relu'
hyperparameters['conv_activation_func8'] = 'relu'
hyperparameters['conv_activation_func9'] = 'relu'
hyperparameters['conv_activation_func10'] = 'relu'
hyperparameters['conv_activation_func11'] = 'relu'
hyperparameters['conv_activation_func12'] = 'relu'
hyperparameters['conv_activation_func13'] = 'relu'
hyperparameters['conv_activation_func14'] = 'relu'
hyperparameters['conv_activation_func15'] = 'relu'
hyperparameters['conv_init_mode1'] = 'glorot_uniform'
hyperparameters['conv_init_mode2'] = 'glorot_uniform'
hyperparameters['conv_init_mode3'] = 'glorot_uniform'
hyperparameters['conv_init_mode4'] = 'glorot_uniform'
hyperparameters['conv_init_mode5'] = 'glorot_uniform'
hyperparameters['conv_init_mode6'] = 'glorot_uniform'
hyperparameters['conv_init_mode7'] = 'glorot_uniform'
hyperparameters['conv_init_mode8'] = 'glorot_uniform'
hyperparameters['conv_init_mode9'] = 'glorot_uniform'
hyperparameters['conv_init_mode10'] = 'glorot_uniform'
hyperparameters['conv_init_mode11'] = 'glorot_uniform'
hyperparameters['conv_init_mode12'] = 'glorot_uniform'
hyperparameters['conv_init_mode13'] = 'glorot_uniform'
hyperparameters['conv_init_mode14'] = 'glorot_uniform'
hyperparameters['conv_init_mode15'] = 'glorot_uniform'
hyperparameters['conv_weight_constraint1'] = 3
hyperparameters['conv_weight_constraint2'] = 3
hyperparameters['conv_weight_constraint3'] = 3
hyperparameters['conv_weight_constraint4'] = 3
hyperparameters['conv_weight_constraint5'] = 3
hyperparameters['conv_weight_constraint6'] = 3
hyperparameters['conv_weight_constraint7'] = 3
hyperparameters['conv_weight_constraint8'] = 3
hyperparameters['conv_weight_constraint9'] = 3
hyperparameters['conv_weight_constraint10'] = 3
hyperparameters['conv_weight_constraint11'] = 3
hyperparameters['conv_weight_constraint12'] = 3
hyperparameters['conv_weight_constraint13'] = 3
hyperparameters['conv_weight_constraint14'] = 3
hyperparameters['conv_weight_constraint15'] = 3
hyperparameters['pool_size1'] = 5
hyperparameters['pool_size2'] = 5
hyperparameters['pool_size3'] = 5
hyperparameters['pool_size4'] = 5
hyperparameters['pool_size5'] = 5
hyperparameters['pool_size6'] = 5
hyperparameters['pool_size7'] = 5
hyperparameters['pool_size8'] = 5
hyperparameters['pool_size9'] = 5
hyperparameters['pool_size10'] = 5
hyperparameters['pool_size11'] = 5
hyperparameters['pool_size12'] = 5
hyperparameters['pool_size13'] = 5
hyperparameters['pool_size14'] = 5
hyperparameters['dropout_rate1'] = 0.2
hyperparameters['neurons1'] = 1
hyperparameters['neurons2'] = 1
hyperparameters['neurons3'] = 1
hyperparameters['neurons4'] = 1
hyperparameters['neurons5'] = 1
hyperparameters['neurons6'] = 1
hyperparameters['neurons7'] = 1
hyperparameters['neurons8'] = 1
hyperparameters['neurons9'] = 1
hyperparameters['neurons10'] = 1
hyperparameters['neurons11'] = 1
hyperparameters['neurons12'] = 1
hyperparameters['neurons13'] = 1
hyperparameters['neurons14'] = 1
hyperparameters['neurons15'] = 1
hyperparameters['dense_activation_func1'] = 'relu'
hyperparameters['dense_activation_func2'] = 'relu'
hyperparameters['dense_activation_func3'] = 'relu'
hyperparameters['dense_activation_func4'] = 'relu'
hyperparameters['dense_activation_func5'] = 'relu'
hyperparameters['dense_activation_func6'] = 'relu'
hyperparameters['dense_activation_func7'] = 'relu'
hyperparameters['dense_activation_func8'] = 'relu'
hyperparameters['dense_activation_func9'] = 'relu'
hyperparameters['dense_activation_func10'] = 'relu'
hyperparameters['dense_activation_func11'] = 'relu'
hyperparameters['dense_activation_func12'] = 'relu'
hyperparameters['dense_activation_func13'] = 'relu'
hyperparameters['dense_activation_func14'] = 'relu'
hyperparameters['dense_activation_func15'] = 'relu'
hyperparameters['dense_init_mode1'] = 'glorot_uniform'
hyperparameters['dense_init_mode2'] = 'glorot_uniform'
hyperparameters['dense_init_mode3'] = 'glorot_uniform'
hyperparameters['dense_init_mode4'] = 'glorot_uniform'
hyperparameters['dense_init_mode5'] = 'glorot_uniform'
hyperparameters['dense_init_mode6'] = 'glorot_uniform'
hyperparameters['dense_init_mode7'] = 'glorot_uniform'
hyperparameters['dense_init_mode8'] = 'glorot_uniform'
hyperparameters['dense_init_mode9'] = 'glorot_uniform'
hyperparameters['dense_init_mode10'] = 'glorot_uniform'
hyperparameters['dense_init_mode11'] = 'glorot_uniform'
hyperparameters['dense_init_mode12'] = 'glorot_uniform'
hyperparameters['dense_init_mode13'] = 'glorot_uniform'
hyperparameters['dense_init_mode14'] = 'glorot_uniform'
hyperparameters['dense_init_mode15'] = 'glorot_uniform'
hyperparameters['dense_weight_constraint1'] = 3
hyperparameters['dense_weight_constraint2'] = 3
hyperparameters['dense_weight_constraint3'] = 3
hyperparameters['dense_weight_constraint4'] = 3
hyperparameters['dense_weight_constraint5'] = 3
hyperparameters['dense_weight_constraint6'] = 3
hyperparameters['dense_weight_constraint7'] = 3
hyperparameters['dense_weight_constraint8'] = 3
hyperparameters['dense_weight_constraint9'] = 3
hyperparameters['dense_weight_constraint10'] = 3
hyperparameters['dense_weight_constraint11'] = 3
hyperparameters['dense_weight_constraint12'] = 3
hyperparameters['dense_weight_constraint13'] = 3
hyperparameters['dense_weight_constraint14'] = 3
hyperparameters['dense_weight_constraint15'] = 3
hyperparameters['output_init_mode'] = 'glorot_uniform'
# hyperparameters['layers'] = ['embedding_layer', 'convolutional_layer', 
#                              'global_maxpooling_layer','dense_layer', 
#                              'dropout_layer', 'output_layer']

# object class
util = utility()

# Read data
df = util.read_CSV(datasetPath + training_path)

from sklearn.model_selection import train_test_split
dfTraining, dfVal = train_test_split(df, test_size = 0.3, random_state=42)

# get texts and labels
textsTraining, y_train = util.get_text_label(dfTraining)
textsVal, y_val = util.get_text_label(dfVal)

# Tokenize words
tokenizer = util.tokenize_texts(textsTraining)
X_train = tokenizer.texts_to_sequences(textsTraining)
X_val = tokenizer.texts_to_sequences(textsVal)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# get maxlen
maxlen = util.get_max_length_of_sentences(textsTraining)

# Pad sequences with zeros
X_train = util.padding_texts(X_train, maxlen)
X_val = util.padding_texts(X_val, maxlen)


# Create Testing Results
f = open(resultsPath + testing_name + ".csv", "a+")
f.write("i,accuracy,precision,recall,f1Score,time,epochs,batch_size,optimizer,learning_rate,layers\n")
f.close()

for i in range(len(combination)):
    cnn = CNN()
    hyperparameters['epochs'] = combination[i][0]
    hyperparameters['batch_size'] = combination[i][1]
    hyperparameters['optimizer'] = combination[i][2]
    hyperparameters['learning_rate'] = combination[i][3]
    hyperparameters['layers'] = combination[i][4]

    if int(hyperparameters['output_dim']) == 50:
        glove = glovePath[0]
    elif int(hyperparameters['output_dim']) == 100:
        glove = glovePath[1]
    elif int(hyperparameters['output_dim']) == 200:
        glove = glovePath[2]
    elif int(hyperparameters['output_dim']) == 300:
        glove = glovePath[3]

    embedding_matrix = util.create_embedding_matrix(glove, tokenizer.word_index, int(hyperparameters['output_dim']))

    then = time.time()
    # grid search
    model = cnn.cnn_model(vocab_size, maxlen, embedding_matrix, hyperparameters, hyperparameters['layers'])

    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', verbose=False, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(archPath + testing_name + str(i + 1) + ".h5", monitor='val_f1_score', mode='max', verbose=False, save_best_only=True)]
    
    model.summary()
    y_train = np.uint8(y_train)
    y_val = np.uint8(y_val)
    model.fit(X_train, y_train, epochs=hyperparameters['epochs'], verbose=False, validation_data=(X_val, y_val),
              batch_size=hyperparameters['batch_size'], callbacks=callbacks)
    
    # save model to a file
    # model.save(archPath + testing_name + str(i + 1) + ".h5")

    dependencies = {
    'f1_score': util.f1_score
    }

    # load the saved model
    saved_model = tf.keras.models.load_model(archPath + testing_name + str(i + 1) + ".h5", custom_objects=dependencies)   

    y_pred = saved_model.predict_classes(X_val)    

    # CNN metrics
    accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = util.get_testing_metric(y_val, y_pred)

    now = time.time()
    diff = now - then
    print(diff)
    print(f1ScoreBinary)

    # save testing data
    f = open(resultsPath + testing_name + ".csv", 'a')
    f.write(str(i + 1)
            + ',' + str(accuracyScore)
            + ',' + str(precisionScoreBinary)
            + ',' + str(recallScoreBinary)
            + ',' + str(f1ScoreBinary)
            + ',' + str(diff)
            + ',' + str(hyperparameters['epochs'])
            + ',' + str(hyperparameters['batch_size'])
            + ',' + str(hyperparameters['optimizer'])
            + ',' + str(hyperparameters['learning_rate'])
            + ',' + str(hyperparameters['layers'])
            + '\n')
    f.close()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 73, 100)           1360200   
_________________________________________________________________
conv1d (Conv1D)              (None, 64, 98)            14080     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12, 98)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 96)            2368      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 96)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 94)            2368      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 12, 94)            0

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


33.800774574279785
0.0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 73, 100)           1360200   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 64, 98)            14080     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 12, 98)            0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 64, 96)            2368      
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 12, 96)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 64, 94)            2368      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (N

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


31.797014713287354
0.0
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 73, 100)           1360200   
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 64, 98)            14080     
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 12, 98)            0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 64, 96)            2368      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 12, 96)            0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 64, 94)            2368      
_________________________________________________________________
max_pooling1d_30 (MaxPooling (N

...

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


33.070815563201904
0.0
Model: "sequential_298"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_298 (Embedding)    (None, 73, 100)           1360200   
_________________________________________________________________
conv1d_4470 (Conv1D)         (None, 64, 98)            14080     
_________________________________________________________________
max_pooling1d_4172 (MaxPooli (None, 12, 98)            0         
_________________________________________________________________
conv1d_4471 (Conv1D)         (None, 64, 96)            2368      
_________________________________________________________________
max_pooling1d_4173 (MaxPooli (None, 12, 96)            0         
_________________________________________________________________
conv1d_4472 (Conv1D)         (None, 64, 94)            2368      
_________________________________________________________________
max_pooling1d_4174 (MaxPooli 

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


35.29082274436951
0.0
Model: "sequential_299"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_299 (Embedding)    (None, 73, 100)           1360200   
_________________________________________________________________
conv1d_4485 (Conv1D)         (None, 64, 98)            14080     
_________________________________________________________________
max_pooling1d_4186 (MaxPooli (None, 12, 98)            0         
_________________________________________________________________
conv1d_4486 (Conv1D)         (None, 64, 96)            2368      
_________________________________________________________________
max_pooling1d_4187 (MaxPooli (None, 12, 96)            0         
_________________________________________________________________
conv1d_4487 (Conv1D)         (None, 64, 94)            2368      
_________________________________________________________________
max_pooling1d_4188 (MaxPooli (

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
